In [1]:
# import packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import math
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
from scipy import optimize
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit
from scipy.interpolate import interpn
from scipy.interpolate import interp1d

import tkinter as tk
import tkinter.filedialog
import tkinter.messagebox
import os
import glob
import time

from ast import literal_eval


### Execute the next cell if you have a jupyter dark theme
Don't if you don't or don't know what this means

https://medium.com/datadriveninvestor/how-can-i-customize-jupyter-notebook-into-dark-mode-7985ce780f38

In [2]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

#restart the core in case you executed the cell in white theme and cannot see ticks anymore

In [3]:
  
def gaussian(x, height, center, width):
    return height*np.exp(-(x - center)**2/(2*width**2))

def gaussian_fit_simple_results(data, oversampling = 10, guessed_width = 20):
    
    #########
    #########fit gaussian in left window to find center of band
    #########
    min_x =  np.min(data)
    max_x =  np.max(data)
    if math.isnan(max_x-min_x) or round((max_x-min_x)*oversampling)==0:
        pars, hData_filtered, hData_filtered_x = [], [], []
    else:  
        bins_single = int(round((max_x-min_x)*oversampling))
        # histogram of positions along the x axis
        h_filtered = np.histogram(data)#, bins = bins_single)  
        #histogram height
        hData_filtered = h_filtered[0]
        ###get the center of each bin
        # bin size
        step = (h_filtered[1][1]-h_filtered[1][0])
        hData_filtered_x = h_filtered[1][:len(h_filtered[1])-1]+step/2
        #guess that band is centered on the window
        guess = [np.mean(hData_filtered), np.mean(hData_filtered_x), guessed_width]
        
        try:
            pars, cov = curve_fit(f=gaussian, xdata=hData_filtered_x, ydata=hData_filtered, p0=guess, bounds=(-np.inf, np.inf))
        except:
            pars = []
    return pars, hData_filtered, hData_filtered_x

## Load result of analysis

In [4]:
root = tk.Tk()
loc_filename = tk.filedialog.askopenfilename(parent=root, 
                                             title='Please select the file containing the result of your analysis')
root.withdraw()

all_results = pd.read_csv(loc_filename)

#create a new column to estimate sarcomere length later on
sarcomere_lengths = [0.0]*len(all_results)
all_results['sarcomere_length'] = sarcomere_lengths
all_results.head(3)

,day,fly_sample,nanobody,fly_number,selection,probe,dye,zdisk,band_distance,band_positions,band_widths,band_widths_estimate,sarcomere_length
0,200818,1,Nano 42,1,6,P1,Atto643,2,206.748490,"[32.867915486650446, 34.458288488683436]","[0.21851438896533762, 0.22615895125313176]","[0.22240971972258616, 0.2279757313209527]",0.0
1,200818,1,Nano 42,1,6,P1,Atto643,3,216.613983,"[60.533437572902734, 62.19969898373392]","[0.23683047815752153, 0.24357284523467207]","[0.16570896401710278, 0.2379513937915265]",0.0
2,200818,1,Nano 42,1,7,P1,Atto643,1,204.493419,"[7.661997634895327, 9.235023931065665]","[0.4396865056549227, 0.24527892422835432]","[0.3556327089229639, 0.2308696421266542]",0.0


## Estimate of sarcomere length

In [5]:
samples = all_results.fly_sample.unique()
FOVs = all_results.fly_number.unique()
nanobodies = all_results.nanobody.unique()
selections = all_results.selection.unique()
days = all_results.day.unique()
  
for n in days:
    cond5 = all_results['day'] == n
    for i in samples:
        cond1 = all_results['fly_sample'] == i
        for j in FOVs:
            cond2 = all_results['fly_number'] == j
            for k in nanobodies:
                cond3 = all_results['nanobody'] == k
                for l in selections:
                    cond4 = all_results['selection'] == l
                    selection = all_results[cond1 & cond2 & cond3 & cond4 & cond5]
                    positions = selection['band_positions'].apply(literal_eval).to_numpy()
                    zdisks = selection['zdisk']
                    if len(zdisks) > 2:
                        indices = all_results[cond1 & cond2 & cond3 & cond4 & cond5].index
                        sarcomere_length = []
                        for m in range(len(zdisks)-2):
                            #print(m)
                            left = (positions[m][0]+positions[m][1])/2
                            right = (positions[m+2][0]+positions[m+2][1])/2
                            sarcomere_length = 130*(right-left)/2
                            #print(type(sarcomere_length))
                            all_results.at[indices[m+1], 'sarcomere_length'] = sarcomere_length
                
all_results.head(10)

,day,fly_sample,nanobody,fly_number,selection,probe,dye,zdisk,band_distance,band_positions,band_widths,band_widths_estimate,sarcomere_length
0,200818,1,Nano 42,1,6,P1,Atto643,2,206.748490,"[32.867915486650446, 34.458288488683436]","[0.21851438896533762, 0.22615895125313176]","[0.22240971972258616, 0.2279757313209527]",0.000000
1,200818,1,Nano 42,1,6,P1,Atto643,3,216.613983,"[60.533437572902734, 62.19969898373392]","[0.23683047815752153, 0.24357284523467207]","[0.16570896401710278, 0.2379513937915265]",0.000000
2,200818,1,Nano 42,1,7,P1,Atto643,1,204.493419,"[7.661997634895327, 9.235023931065665]","[0.4396865056549227, 0.24527892422835432]","[0.3556327089229639, 0.2308696421266542]",0.000000
3,200818,1,Nano 42,1,7,P1,Atto643,2,187.637090,"[36.381442767586144, 37.82480499872456]","[0.2516178183522468, 0.2151535553483115]","[0.21530918040950206, 0.1978635282552481]",3712.464804
4,200818,1,Nano 42,1,7,P1,Atto643,3,193.426990,"[64.81940395432957, 66.30730387812818]","[0.18249507172917895, 0.2081670817635209]","[0.17913721532571136, 0.20948512962670043]",3685.536838
5,200818,1,Nano 42,1,7,P1,Atto643,4,213.577221,"[92.98223977227072, 94.62514147245776]","[0.2094361801242504, 0.19724862698471987]","[0.2700776978033651, 0.21740176036158912]",0.000000
6,200818,1,Nano 42,1,8,P1,Atto643,2,210.757626,"[38.86207462522417, 40.48328713415544]","[0.19762387379174462, 0.2295100701289125]","[0.2300581127532284, 0.24829400224914694]",0.000000
7,200818,1,Nano 42,1,8,P1,Atto643,3,199.678112,"[67.06555442556487, 68.60153990074515]","[0.24906985406773433, 0.22358487941369987]","[0.23270120087123433, 0.2871804383972764]",3683.278503
8,200818,1,Nano 42,1,8,P1,Atto643,4,202.254311,"[95.56060280886564, 97.11640519949574]","[0.24503221380647247, 0.24961249192530768]","[0.248226096224383, 0.19385759304704006]",0.000000
9,200818,1,Nano 42,1,10,P1,Atto643,2,180.161765,"[31.993829127091963, 33.379688858717856]","[0.23849994663926954, 0.2939537208401544]","[0.2481865963917902, 0.2224873675525938]",0.000000


In [8]:
# Save results with sarcomere length to avoid extracting sarcomere length each time
filename_with_sarc = r'C:\Users\pierr\Desktop\Sarcomere simulation\All_results_with_sarc_length.csv'
all_results.to_csv(filename_with_sarc)